In [20]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import evaluate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import (BinaryAccuracy, FalsePositives, FalseNegatives, TruePositives,
                                       TrueNegatives, Precision, Recall, AUC, binary_accuracy,Accuracy,
                                       TopKCategoricalAccuracy, CategoricalAccuracy,SparseCategoricalAccuracy)
from tensorflow.keras.optimizers import Adam
from datasets import load_dataset
from transformers import (BertTokenizerFast,TFBertTokenizer,BertTokenizer,RobertaTokenizerFast,DataCollatorForTokenClassification,
                          DataCollatorWithPadding,TFRobertaForSequenceClassification,TFBertForSequenceClassification,
                          TFBertModel,create_optimizer,TFRobertaForTokenClassification,TFAutoModelForTokenClassification,)

In [2]:
BATCH_SIZE=8
NUM_EPOCHS=2

## Data Prep

In [3]:
dataset = load_dataset("conll2003")

In [4]:
model_id="roberta-base"
tokenizer=RobertaTokenizerFast.from_pretrained(model_id,add_prefix_space=True)

In [5]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [6]:
def tokenizer_function(dataset):
  out=tokenizer(dataset["tokens"],truncation=True,is_split_into_words=True,)
  out['labels']=align_labels_with_tokens(dataset["ner_tags"],out.word_ids())
  return out

In [7]:
tokenized_dataset=dataset.map(tokenizer_function,remove_columns=['id','tokens','pos_tags','chunk_tags','ner_tags',])

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

In [8]:
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, return_tensors="tf"
)

In [9]:
tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    collate_fn=data_collator,
    shuffle=True,
    batch_size=BATCH_SIZE,
)

2024-07-31 19:45:23.924684: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-31 19:45:23.931421: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-31 19:45:23.931579: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-31 19:45:23.943609: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-31 19:45:23.943713: I external/local_xla/xla/stream_executor

In [10]:
tf_val_dataset = tokenized_dataset["validation"].to_tf_dataset(
    collate_fn=data_collator,
    shuffle=False,
    batch_size=BATCH_SIZE,
)

## Modeling

In [11]:
model=TFRobertaForTokenClassification.from_pretrained(
    model_id,
    num_labels=9,
)

2024-07-31 19:45:27.733317: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 154414080 exceeds 10% of free system memory.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForTokenClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN 

In [12]:
model.summary()

Model: "tf_roberta_for_token_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLaye  multiple                  124055040 
 r)                                                              
                                                                 
 dropout_37 (Dropout)        multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  6921      
                                                                 
Total params: 124061961 (473.26 MB)
Trainable params: 124061961 (473.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Training

In [13]:
batches_per_epoch = len(tokenized_dataset["train"]) // BATCH_SIZE
total_train_steps = int(batches_per_epoch*NUM_EPOCHS)

In [14]:
optimizer, schedule = create_optimizer(init_lr=2e-5,num_warmup_steps=0, num_train_steps=total_train_steps,)

2024-07-31 19:45:28.345686: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [15]:
model.compile(optimizer=optimizer,)
    #metrics=["accuracy"])

In [16]:
history=model.fit(
    tf_train_dataset,
    validation_data=tf_val_dataset,
    verbose = 1,
    epochs=NUM_EPOCHS,)

Epoch 1/2


2024-07-31 19:45:57.857946: I external/local_xla/xla/service/service.cc:168] XLA service 0x7ff4810f4a20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-31 19:45:57.858017: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-07-31 19:45:57.884687: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-31 19:45:57.939746: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1722435358.040234  119019 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1756/1756 [==============================] - 404s 205ms/step - loss: 0.1270 - val_loss: 0.0501
Epoch 2/2
1756/1756 [==============================] - 331s 188ms/step - loss: 0.0400 - val_loss: 0.0427


## Evaluation

In [21]:
metric=evaluate.load("seqeval")

In [22]:
ind_to_label={0:'O', 1:'B-PER',2:'I-PER',3:'B-ORG',4:'I-ORG',5:'B-LOC',6:'I-LOC',7:'B-MISC',8:'I-MISC'}
all_predictions = []
all_labels = []

In [23]:
for batch in tf_val_dataset:
  logits = model.predict(batch)["logits"]
  labels = batch["labels"].numpy()
  predictions = tf.argmax(logits, axis=-1).numpy()
  #print(labels)
  #print(predictions)
  for prediction, label in zip(predictions, labels):
    for predicted_idx, label_idx in zip(prediction, label):
      if label_idx == -100:
          continue
      all_predictions.append(ind_to_label[predicted_idx])
      all_labels.append(ind_to_label[label_idx])

1/1 [==============================] - 0s 87ms/step


In [24]:
metric.compute(predictions=[all_predictions], references=[all_labels])

{'LOC': {'precision': 0.9658721560130011,
  'recall': 0.9706042460533478,
  'f1': 0.9682324192234592,
  'number': 1837},
 'MISC': {'precision': 0.8511722731906218,
  'recall': 0.9056399132321041,
  'f1': 0.8775617446137678,
  'number': 922},
 'ORG': {'precision': 0.9031091829356471,
  'recall': 0.9313944817300522,
  'f1': 0.9170337738619677,
  'number': 1341},
 'PER': {'precision': 0.9719676549865229,
  'recall': 0.9788273615635179,
  'f1': 0.9753854476602651,
  'number': 1842},
 'overall_precision': 0.9348722176422094,
 'overall_recall': 0.9542241669471558,
 'overall_f1': 0.9444490713750313,
 'overall_accuracy': 0.9884630726274125}

## Testing

In [25]:
inputs=tokenizer(["Hello John, what do you think of Iceland"], padding=True,return_tensors="tf")

In [26]:
print(inputs.tokens())
print(inputs.word_ids())
print(inputs['input_ids'])

['<s>', 'ĠHello', 'ĠJohn', ',', 'Ġwhat', 'Ġdo', 'Ġyou', 'Ġthink', 'Ġof', 'ĠIceland', '</s>']
[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]
tf.Tensor([[    0 20920   610     6    99   109    47   206     9 14605     2]], shape=(1, 11), dtype=int32)


In [27]:
logits = model(**inputs).logits
print(logits.shape)
print(tf.argmax(logits,axis=-1))

(1, 11, 9)
tf.Tensor([[0 0 1 0 0 0 0 0 0 5 0]], shape=(1, 11), dtype=int64)


In [28]:
ind_to_label={0:'O', 1:'B-PER',2:'I-PER',3:'B-ORG',4:'I-ORG',5:'B-LOC',6:'I-LOC',7:'B-MISC',8:'I-MISC'}
out_str=""
current_index=0

In [29]:
for i in range(1,len(inputs.tokens())-1):
  if tf.argmax(logits,axis=-1)[0][i]!=0:
    out_str+=" "+str(inputs.tokens()[i])+"--->"+str(ind_to_label[tf.argmax(logits,axis=-1).numpy()[0][i]])
  else:
    out_str+=" "+str(inputs.tokens()[i])

In [30]:
print(out_str.replace("Ġ",""))

 Hello John--->B-PER , what do you think of Iceland--->B-LOC
